In [0]:
import numpy as np
from glob import glob
from google.colab import drive
drive.mount('/content/drive')
import pickle
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import skimage
import sys, time
from sklearn.model_selection import train_test_split

from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras import Model,Sequential
import keras
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Using TensorFlow backend.


In [0]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/TrainDataset.zip", 'r')
zip_ref.extractall("/content/Training")
zip_ref.close()

In [0]:
zip_ref_test = zipfile.ZipFile("/content/drive/My Drive/Project_C2_Testing-20191129T063942Z-001.zip", 'r')
zip_ref_test.extractall("/content/test")
zip_ref_test.close()

In [0]:
TRAIN_PATH = "/content/Training/TrainDataset"
TEST_PATH = "/content/test/Project_C2_Testing"

In [0]:
def load_data():
  path = TRAIN_PATH
  df = pd.read_csv('train_files.csv')
  X = []
  y = []
  # org = []
  for i, row in df.iterrows():
    filename = row["file_name"] 
    label = row["annotation"]
    file = open('/content/Training/TrainDataset/'+filename,'rb')
    im = cv2.imread('/content/Training/TrainDataset/'+filename)
    X.append(im)
    y.append(label)
  X = np.array(X)
  y = np.array(y)
  print(X.shape)
  
  return X,y
  
X,y = load_data()

(896, 480, 640, 3)


In [0]:
trainX, valX, trainY, valY = train_test_split(X,y,test_size=0.3, stratify=y)

In [0]:
model_resnet = keras.applications.resnet.ResNet152(include_top=False, weights='imagenet',input_tensor=None, input_shape=None, pooling=None)
output = model_resnet.output
resnet_optimized_model = Model(input=model_resnet.input, output=output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
resnet_trainFeatures=resnet_optimized_model.predict(trainX)
resnet_valFeatures=resnet_optimized_model.predict(valX)
resnet_trainFeatures.shape

(627, 15, 20, 2048)

In [0]:
pickle.dump(resnet_trainFeatures,open('trainFeatures.pkl','wb'))
pickle.dump(resnet_valFeatures,open('valFeatures.pkl','wb'))
pickle.dump(trainY,open('trainLabels.pkl','wb'))
pickle.dump(valY,open('valLabels.pkl','wb'))

In [0]:
trainFeatures = pickle.load(open('trainFeatures.pkl','rb'))
valFeatures = pickle.load(open('valFeatures.pkl','rb'))
trainLabels = pickle.load(open('trainLabels.pkl','rb'))
valLabels = pickle.load(open('valLabels.pkl','rb'))
print(trainFeatures.shape,trainLabels.shape)
print(valFeatures.shape,valLabels.shape)

(627, 15, 20, 2048) (627,)
(269, 15, 20, 2048) (269,)


In [0]:
#parameters
batch_size = [32,64,128,256,627]
lr= [0.01,0.001,0.0001]
epochs=[10,15,20,25,30,50,100,200,300,500]
drop=[0,0.2,0.3,0.5]
opt = ['adam','adagrad','sgd','rmsprop','adadelta']
act = [None,'relu','tanh','sigmoid']
hiddenunits = [64,128,256,512,1024,2048,2046,4096]

param_grid = dict(lr=lr,epochs=epochs,drop=drop,batch_size=batch_size,opt=opt,act=act,hiddenunits=hiddenunits)

In [0]:
transformedX = np.concatenate((trainFeatures,valFeatures))
transformedY = np.concatenate((trainLabels,valLabels))

In [0]:
def create_model(act=None, lr=0.001, drop=0.5, opt='adam', hiddenunits=64):
  model = Sequential()
  model.add(GlobalAveragePooling2D(input_shape=(15, 20, 2048)))
  if type(hiddenunits) == type(2):
    model.add(Dense(hiddenunits, activation=act))
    if drop > 0:
      model.add(Dropout(drop))
  else:
    model.add(Dense(hiddenunits[0], activation=act))
    if drop > 0:
      model.add(Dropout(drop))
    model.add(Dense(hiddenunits[1], activation=act))
    if drop > 0:
      model.add(Dropout(drop))
    
  model.add(Dense(5, activation='softmax'))
  
  if opt == 'adam':
    optimizer = keras.optimizers.Adam(lr=lr)
  elif opt == 'sgd':
    optimizer = keras.optimizers.SGD(lr=lr)
  elif opt == 'rmsprop':
    optimizer = keras.optimizers.RMSprop(lr=lr)
  elif opt == 'adadelta':
    optimizer = keras.optimizers.Adadelta(lr=lr)
  elif opt == 'adagrad':
    optimizer = keras.optimizers.Adagrad(lr=lr)
  model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=1)

In [0]:
cv=3
#ss = StratifiedShuffleSplit(n_splits=cv, test_size=0.3, random_state=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=10, cv=cv, verbose=10)  #GridSearch with 3-fold Cross Validation
grid_result = grid.fit(transformedX, transformedY)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96000 candidates, totalling 288000 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


TerminatedWorkerError: ignored

In [0]:
!pip install scikit-optimize

     |████████████████████████████████| 81kB 4.8MB/s 


In [0]:
# Importing Libraries

import tensorflow
from tensorflow.python.keras import backend 

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers
from keras.utils import np_utils
from keras.optimizers import Adam

import pandas as pd
import matplotlib.pylab as plt

import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

import gzip
import os
import sys
import struct
import numpy as np

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
dim_learning_rate = Real(low=1e-4, high=1e-1, prior='log-uniform',
                         name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=3, name='num_dense_layers')
dim_num_input_nodes = Integer(low=32, high=2048, name='num_input_nodes')
dim_num_dense_nodes = Integer(low=32, high=1024, name='num_dense_nodes')
dim_activation = Categorical(categories=['tanh','sigmoid','relu', 'softmax'],name='activation')
dim_batch_size = Integer(low=64, high=128, name='batch_size')
dim_adam_decay = Real(low=1e-6,high=1e-2,name="adam_decay")

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_input_nodes,
              dim_num_dense_nodes,
              dim_activation,
              dim_batch_size,
              dim_adam_decay
             ]
default_parameters = [1e-2, 1,32, 128, 'relu',64, 1e-6]

In [0]:
# Model for Hyperparameter optimization

from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPool2D, Dense, Dropout, Flatten, GlobalAveragePooling2D

def build_model(learning_rate, num_dense_layers,num_input_nodes,num_dense_nodes, 
                 activation, adam_decay):
  model = Sequential()
  # add Convolutional layers
  # model.add(Conv2D(filters=32, kernel_size=(3,3), activation=activation, padding='same',
  #                    input_shape=(28,28,1)))
  # model.add(MaxPooling2D(pool_size=(2,2)))
  # model.add(Conv2D(filters=64, kernel_size=(3,3), activation=activation, padding='same'))
  # model.add(MaxPooling2D(pool_size=(2,2)))
  # model.add(Conv2D(filters=64, kernel_size=(3,3), activation=activation, padding='same'))
  # model.add(MaxPooling2D(pool_size=(2,2)))
  # model.add(Flatten())
  model.add(GlobalAveragePooling2D(input_shape=(15, 20, 2048)))
  # Densely connected layers
  model.add(Dense(num_input_nodes, activation=activation))
  for i in range(num_dense_layers):
        name = 'layer_dense_{0}'.format(i+1)
        model.add(Dense(num_dense_nodes,
                 activation=activation,
                        name=name
                 ))
  model.add(Dense(5,activation='softmax',name="output_layer"))
  #setup our optimizer and compile
  adam = Adam(lr=learning_rate, decay= adam_decay)
  model.compile(optimizer = adam, loss='sparse_categorical_crossentropy', metrics=["accuracy"])
  return model

In [0]:
# Perform optimization

@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_input_nodes, 
            num_dense_nodes,activation, batch_size,adam_decay):

    model = build_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_input_nodes=num_input_nodes,
                         num_dense_nodes=num_dense_nodes,
                         activation=activation,
                         adam_decay=adam_decay
                        )
    

    #named blackbox becuase it represents the structure
#     print(trainFeatures.shape,trainLabels.shape)
# print(valFeatures.shape,valLabels.shape)
    blackbox = model.fit(x=trainFeatures,
                        y=trainLabels,
                        epochs=3,
                        batch_size=batch_size,
                        validation_data=(valFeatures, valLabels)
                        )
    
    #return the validation accuracy for the last epoch.
    count = 0
    train_accuracy = blackbox.history['acc'][-1]
    train_loss = blackbox.history['loss'][-1]
    val_accuracy = blackbox.history['val_acc'][-1]
    val_loss = blackbox.history['val_loss'][-1]
  
    # Print the classification accuracy.
    print("Training Accuracy: {0:.2%}".format(train_accuracy))
    print("Training Loss: {0:.2%}".format(train_loss))
    print("Validation Accuracy: {0:.2%}".format(val_accuracy))
    print("Validation Loss: {0:.2%}".format(val_loss))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    backend.clear_session()
    tensorflow.reset_default_graph()
    
    return val_accuracy

In [0]:
backend.clear_session()
tensorflow.reset_default_graph()

In [0]:
gp_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=30,
                            noise= 0.01,
                            n_jobs=-1,
                            kappa = 5,
                            x0=default_parameters)

Train on 627 samples, validate on 269 samples
Epoch 1/3
627/627 [==============================] - 1s 2ms/step - loss: 0.4714 - acc: 0.7751 - val_loss: 0.7811 - val_acc: 0.6803
Epoch 2/3
627/627 [==============================] - 1s 2ms/step - loss: 0.5434 - acc: 0.7352 - val_loss: 0.8797 - val_acc: 0.6171
Epoch 3/3
627/627 [==============================] - 1s 2ms/step - loss: 0.4576 - acc: 0.7863 - val_loss: 0.8573 - val_acc: 0.7063
Training Accuracy: 78.63%
Training Loss: 45.76%
Validation Accuracy: 70.63%
Validation Loss: 85.73%

Train on 627 samples, validate on 269 samples
Epoch 1/3
627/627 [==============================] - 1s 2ms/step - loss: 0.4555 - acc: 0.7703 - val_loss: 0.8352 - val_acc: 0.6877
Epoch 2/3
627/627 [==============================] - 1s 2ms/step - loss: 0.4121 - acc: 0.8086 - val_loss: 0.7899 - val_acc: 0.6914
Epoch 3/3
627/627 [==============================] - 1s 2ms/step - loss: 0.3923 - acc: 0.8309 - val_loss: 0.8236 - val_acc: 0.6914
Training Accuracy: 83

In [0]:
# Various Hyper Parameters and Accuracy achieved

new_dataframe = [pd.DataFrame(gp_result.x_iters, columns = ["learning rate","hidden layers","input layer nodes","hidden layer nodes",
                                           "activation function","batch size","adam learning rate decay"]),
(pd.Series(gp_result.func_vals*100, name="accuracy"))]
new_dataframe = pd.concat(new_dataframe,axis=1)
new_dataframe

,learning rate,hidden layers,input layer nodes,hidden layer nodes,activation function,batch size,adam learning rate decay,accuracy
0,0.010000,1,32,128,relu,64,0.000001,70.631970
1,0.007657,3,2021,423,relu,101,0.001671,69.144981
2,0.007195,2,1433,292,relu,105,0.007795,65.055764
3,0.091027,2,78,800,sigmoid,87,0.000228,69.888476
4,0.033003,2,1503,581,tanh,115,0.003049,71.003718
5,0.021218,1,598,137,tanh,103,0.007542,67.657993
6,0.003803,1,1427,495,softmax,72,0.004895,67.657992
7,0.044515,2,1661,974,tanh,121,0.003514,69.516729
8,0.017247,2,238,257,relu,90,0.000412,66.914499
9,0.028890,2,806,248,relu,118,0.005248,69.144982
